In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
from datetime import datetime
import pandas as pd
import plotly.express as px

from download import download


mpl.rcParams['figure.figsize'] = (10, 8)
mpl.rcParams['axes.grid'] = False


In [3]:
path = download('https://archive.ics.uci.edu/ml/machine-learning-databases/00501/PRSA2017_Data_20130301-20170228.zip', '/tmp/aq', kind="zip")

Creating data folder...

file_sizes: 7.96MB [00:04, 1.60MB/s]                                            
Extracting zip file...
Successfully downloaded / unzipped to /tmp/aq


In [7]:
!dir /tmp/aq/PRSA_Data_20130301-20170228


Parameter format not correct - "mp".
